<a href="https://colab.research.google.com/github/prgawade/battlefieldofAI/blob/main/session3/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from __future__ import print_function
#PyTorch is an open source machine learning framework
#PyTorch is a Python package that provides two high-level features:
#Tensor computation (like NumPy) with strong GPU acceleration
#Deep neural networks built on a tape-based autograd system
import torch
# A neural networks library deeply integrated with autograd designed for maximum flexibility
import torch.nn as nn
#Convolution functions such as conv1d , conv2d, conv3d
import torch.nn.functional as F
#torch.optim is a package implementing various optimization algorithms.
# Adam - Implements Adam algorithm
# SGD Implements stochastic gradient descent (optionally with momentum).
import torch.optim as optim
# The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.
# Transforms are common image transformations available in the torchvision.transforms module. 
from torchvision import datasets, transforms

In [8]:
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # torch.nn.Conv2d Applies a 2D convolution over an input signal composed of several input planes.
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # input channel is 1 , output channels 32 , number of kernels is 32, size of kernel is 3 X 3
        #28 X 28 X 1 | (3 X 3 X 1) X 32 | 26 X 26 X 32
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 X 28 X 1 # Output - 28 X 28 X 32 # output  RF - 3 
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25)
        )
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input - 28 X 28 X 32 # Output - 28 X 28 X 64 # output  RF - 5
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.25)
        )

        self.selfconv = nn.Conv2d(64, 16, 1, padding=1)

        self.pool1 = nn.MaxPool2d(2, 2) #input - 28 X 28 X 64 # Output - 14 X 14 X 64 # output  RF - 10
        #self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # input 14 X 14 X 64 # Output - 14 X 14 X 128 # output  RF - 12
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25)
        )
        #self.conv4 = nn.Conv2d(32, 64, 3, padding=1) # input 14 X 14 X 128 # Output - 14 X 14 X 256 # output  RF - 14
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.25)
        )
        self.pool2 = nn.MaxPool2d(2, 2) # input 14 X 14 X 256 # Output - 7 X 7 X 256 # output  RF - 28
        self.conv5 = nn.Conv2d(64, 128, 3) # input 7 X 7 X 256 # Output - 5 X 5 X 512 # output  RF - 
        self.conv6 = nn.Conv2d(128, 256, 3)# input 5 X 5 X 512 # Output - 3 X 3 X 1024 # output  RF
        self.conv7 = nn.Conv2d(256, 10, 3) # input 3 X 3 X 1024 # Output - 1 X 1 X 10 # output  RF

    def forward(self, x):
        #x = self.pool1()
        x  = F.relu(self.conv1(x))

        x  = F.relu(self.conv2(x))
        x   = self.selfconv(x)
        x  = self.pool1(x)
        
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x   = self.selfconv(x)
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
"""
  

"\nclass Net(nn.Module):\n    def __init__(self):\n        super(Net, self).__init__()\n        # torch.nn.Conv2d Applies a 2D convolution over an input signal composed of several input planes.\n        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)\n        # input channel is 1 , output channels 32 , number of kernels is 32, size of kernel is 3 X 3\n        #28 X 28 X 1 | (3 X 3 X 1) X 32 | 26 X 26 X 32\n        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 X 28 X 1 # Output - 28 X 28 X 32 # output  RF - 3 \n        self.conv1 = nn.Sequential(\n            nn.Conv2d(1, 32, 3, padding=1),\n            nn.ReLU(),\n            nn.BatchNorm2d(32),\n            nn.Dropout(0.25)\n        )\n        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input - 28 X 28 X 32 # Output - 28 X 28 X 64 # output  RF - 5\n        self.conv2 = nn.Sequential(\n            nn.Conv2d(32, 64

In [9]:
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # torch.nn.Conv2d Applies a 2D convolution over an input signal composed of several input planes.
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # input channel is 1 , output channels 32 , number of kernels is 32, size of kernel is 3 X 3
        #28 X 28 X 1 | (3 X 3 X 1) X 32 | 26 X 26 X 32
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 X 28 X 1 # Output - 28 X 28 X 32 # output  RF - 3 
        self.conv1 = nn.Sequential(   ##input - 28 X 28 X 1 # Output - 26 X 26 X 16 # output  RF - 3 
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25)
        )
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input - 26 X 26 X 32 # Output - 24 X 24 X 64 # output  RF - 5
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25)
        )

        self.selfconv = nn.Conv2d(32, 8, 1, padding=1)

        self.pool1 = nn.MaxPool2d(2, 2) #input - 26 X 28 X 64 # Output - 14 X 14 X 64 # output  RF - 10
        #self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # input 14 X 14 X 64 # Output - 14 X 14 X 128 # output  RF - 12
        self.conv3 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25)
        )
        #self.conv4 = nn.Conv2d(32, 64, 3, padding=1) # input 14 X 14 X 128 # Output - 14 X 14 X 256 # output  RF - 14
        self.conv4 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25)
        )
        self.pool2 = nn.MaxPool2d(2, 2) # input 14 X 14 X 256 # Output - 7 X 7 X 256 # output  RF - 28
        self.conv5 = nn.Conv2d(32, 64, 3) # input 7 X 7 X 256 # Output - 5 X 5 X 512 # output  RF - 
        self.conv6 = nn.Conv2d(64, 128, 3)# input 5 X 5 X 512 # Output - 3 X 3 X 1024 # output  RF
        self.conv7 = nn.Conv2d(128, 10, 3) # input 3 X 3 X 1024 # Output - 1 X 1 X 10 # output  RF

    def forward(self, x):
        #x = self.pool1()
        x  = F.relu(self.conv1(x))

        x  = F.relu(self.conv2(x))
        x   = self.selfconv(x)
        x  = self.pool1(x)
        
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x  = F.relu(self.conv3(x))

        x  = F.relu(self.conv4(x))
        #x   = self.selfconv(x)
        x  = self.pool2(x)
                
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

"""

"\nclass Net(nn.Module):\n    def __init__(self):\n        super(Net, self).__init__()\n        # torch.nn.Conv2d Applies a 2D convolution over an input signal composed of several input planes.\n        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)\n        # input channel is 1 , output channels 32 , number of kernels is 32, size of kernel is 3 X 3\n        #28 X 28 X 1 | (3 X 3 X 1) X 32 | 26 X 26 X 32\n        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 X 28 X 1 # Output - 28 X 28 X 32 # output  RF - 3 \n        self.conv1 = nn.Sequential(   ##input - 28 X 28 X 1 # Output - 26 X 26 X 16 # output  RF - 3 \n            nn.Conv2d(1, 16, 3, padding=1),\n            nn.ReLU(),\n            nn.BatchNorm2d(16),\n            nn.Dropout(0.25)\n        )\n        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input - 26 X 26 X 32 # Output - 24 X 24 X 64 # output  RF - 5\

In [10]:
"""class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # torch.nn.Conv2d Applies a 2D convolution over an input signal composed of several input planes.
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        # input channel is 1 , output channels 32 , number of kernels is 32, size of kernel is 3 X 3
        #28 X 28 X 1 | (3 X 3 X 1) X 32 | 26 X 26 X 32
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 X 28 X 1 # Output - 28 X 28 X 32 # output  RF - 3 
        self.conv1 = nn.Sequential(   ##input - 28 X 28 X 1 # Output - 26 X 26 X 16 # output  RF - 3 
            nn.Conv2d(1, 16, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25)
        )
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.conv2 = nn.Sequential( ##input - 26 X 26 X 16 # Output - 24 X 24 X 32 # output  RF - 5
            nn.Conv2d(16, 32, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25)
        )

        self.selfconv = nn.Conv2d(32, 8, 1, padding=0) ##input - 24 X 24 X 32 # Output - 24 X 24 X 8 # output  RF - 5

        self.pool1 = nn.MaxPool2d(2, 2) #input - 24 X 24 X 8 # Output - 12 X 12 X 8 # output  RF - 10
        #self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # input 14 X 14 X 64 # Output - 14 X 14 X 128 # output  RF - 12
        self.conv3 = nn.Sequential( # input 12 X 12 X 8 # Output - 10 X 10 X 16 # output  RF - 12
            nn.Conv2d(8, 16, 3, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25)
        )
        #self.conv4 = nn.Conv2d(32, 64, 3, padding=1) # input 14 X 14 X 128 # Output - 14 X 14 X 256 # output  RF - 14
        self.conv4 = nn.Sequential( # input 10 X 10 X 16 # Output - 10 X 10 X 32 # output  RF - 14
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25)
        )

             
        self.selfconv = nn.Conv2d(32, 8, 1, padding=0) # input 10 X 10 X 32 # Output - 10 X 10 X 8 # output  RF - 28
        self.pool2 = nn.MaxPool2d(2, 2) # input 10 X 10 X 8 # Output - 5 X 5 X 8 # output  RF - 28
        self.conv5 = nn.Conv2d(8, 16, 3) # input 5 X 5 X 8 # Output - 3 X 3 X 16 # output  RF - 
        #self.conv6 = nn.Conv2d(16, 32, 3)# input 3 X 3 X 16 # Output - 1 X 1 X 32 # output  RF
        #self.conv7 = nn.Conv2d(64, 10, 3) # input 3 X 3 X 1024 # Output - 1 X 1 X 10 # output  RF
        self.conv6 = nn.Sequential( # input 10 X 10 X 16 # Output - 10 X 10 X 32 # output  RF - 14
            nn.Conv2d(16, 32, 3, padding=0),
            nn.ReLU(),
            nn.AvgPool2d((1,1)),
            nn.Dropout(0.25)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(32, 16)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(16, 10)
        )

    def forward(self, x):
        #x = self.pool1()
        x  = F.relu(self.conv1(x))

        x  = F.relu(self.conv2(x))
        x   = self.selfconv(x)
        x  = self.pool1(x)
        
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x  = F.relu(self.conv3(x))

        x  = F.relu(self.conv4(x))
        #x = x.view(-1, 32)
  
        x   = self.selfconv(x)
        x  = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv6(x))
        #x = self.conv6(x)
        #print (x.shape)
        #x = x.view(-1, 64)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = x.view(-1, 10)
        #print (x.shape)
        return F.log_softmax(x)
		"""

"class Net(nn.Module):\n    def __init__(self):\n        super(Net, self).__init__()\n        # torch.nn.Conv2d Applies a 2D convolution over an input signal composed of several input planes.\n        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)\n        # input channel is 1 , output channels 32 , number of kernels is 32, size of kernel is 3 X 3\n        #28 X 28 X 1 | (3 X 3 X 1) X 32 | 26 X 26 X 32\n        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 X 28 X 1 # Output - 28 X 28 X 32 # output  RF - 3 \n        self.conv1 = nn.Sequential(   ##input - 28 X 28 X 1 # Output - 26 X 26 X 16 # output  RF - 3 \n            nn.Conv2d(1, 16, 3, padding=0),\n            nn.ReLU(),\n            nn.BatchNorm2d(16),\n            nn.Dropout(0.25)\n        )\n        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1) \n        self.conv2 = nn.Sequential( ##input - 26 X 26 X 16 # Ou

In [11]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=0,                  
            ),                              
            nn.ReLU(),                      
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2),     
            nn.Dropout(0.25)    
        )

        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 0),     
            nn.ReLU(),                     
            nn.BatchNorm2d(32),
            #nn.MaxPool2d(kernel_size=2), 
            nn.Dropout(0.25), 
        )
        self.selfconv = nn.Conv2d(32, 8, 1, padding=0)
        # fully connected layer, output 10 classes

        self.out1 = nn.Sequential(
            nn.Linear(8 * 8 * 8, 10),      
            nn.Dropout(0.25)
        )
        self.out2 = nn.Sequential(
            nn.Linear(32, 10),          
            nn.Dropout(0.25)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #print(x.shape)
        x = self.selfconv(x)
        x = F.adaptive_avg_pool2d(x, (8, 8))
        #print(x.shape)
        #x = self.conv3(x)
        #print(x.shape)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        #x = self.out1(x)
        output = self.out1(x)
        return  F.log_softmax(output)   # return x for visualization

In [12]:
!pip install torchsummary
#API to view the visualization of the model, which is helpful while debugging your network.
from torchsummary import summary
# Returns a bool indicating if CUDA is currently available.
# NVIDIA’s CUDA is a general purpose parallel computing platform and programming model that accelerates deep learning and other compute-intensive apps by taking advantage of the parallel processing power of GPUs.
use_cuda = torch.cuda.is_available()
# A torch.device is an object representing the device on which a torch.Tensor is or will be allocated.
device = torch.device("cuda" if use_cuda else "cpu")
# Specify which device to use
model = Net().to(device)
# Model summary
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 24, 24]             416
              ReLU-2           [-1, 16, 24, 24]               0
       BatchNorm2d-3           [-1, 16, 24, 24]              32
           Dropout-4           [-1, 16, 24, 24]               0
            Conv2d-5           [-1, 32, 20, 20]          12,800
              ReLU-6           [-1, 32, 20, 20]               0
       BatchNorm2d-7           [-1, 32, 20, 20]              64
         MaxPool2d-8           [-1, 32, 10, 10]               0
           Dropout-9           [-1, 32, 10, 10]               0
           Conv2d-10            [-1, 8, 10, 10]             264
           Linear-11                   [-1, 10]           5,130
          Dropout-12                   [-1, 10]               0
Tota

<ipython-input-11-bc7f884a47af>:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return  F.log_softmax(output)   # return x for visualization


In [13]:

#Sets the seed for generating random numbers. Returns a torch.Generator object.

torch.manual_seed(1)
#batch_size = 128
batch_size = 128
#PyTorch's DataLoader class, which in addition to our Dataset class, also takes in the following important arguments:

#batch_size, which denotes the number of samples contained in each generated batch.
#shuffle. If set to True, we will get a new order of exploration at each pass (or just keep a linear exploration scheme otherwise). Shuffling the order in which examples are fed to the classifier is helpful so that batches between epochs do not look alike. Doing so will eventually make our model more robust.
#num_workers, which denotes the number of processes that generate batches in parallel. A high enough number of workers assures that CPU computations are efficiently managed, i.e. that the bottleneck is indeed the neural network's forward and backward operations on the GPU (and not data generation).
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# Download training data 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, 
                    transform=transforms.Compose([ # Composes several transforms together.
                        transforms.ToTensor(), # Convert a PIL Image or numpy.ndarray to tensor.
                        transforms.Normalize((0.1307,), (0.3081,)) # Normalize(mean, std[, inplace]) Normalize a tensor image with mean and standard deviation.
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [14]:
from tqdm import tqdm
# model defined above
# device in this case will cpu
# train loader to download training images and transform into tensor 
# SGD optimizer
# The learning rate controls how quickly the model is adapted to the problem. Smaller learning rates require more training epochs given the smaller changes made to the weights each update, whereas larger learning rates result in rapid changes and require fewer training epochs.
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # Sets the gradients of all optimized torch.Tensor s to zero.
        output = model(data)
        # loss function
        loss = F.nll_loss(output, target) # loss function - The negative log likelihood loss.
        loss.backward() 
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:

model = Net().to(device)

#torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False, *, maximize=False, foreach=None, differentiable=False)
#params (iterable) – iterable of parameters to optimize or dicts defining parameter groups

#lr (float) – learning rate

#momentum (float, optional) – momentum factor (default: 0)

#weight_decay (float, optional) – weight decay (L2 penalty) (default: 0)

#dampening (float, optional) – dampening for momentum (default: 0)

#nesterov (bool, optional) – enables Nesterov momentum (default: False)

#maximize (bool, optional) – maximize the params based on the objective, instead of minimizing (default: False)

#foreach (bool, optional) – whether foreach implementation of optimizer is used (default: None)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 20
for epoch in range(1, epochs + 1):
    print('epoch value is , ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

# Test set: Average loss: 0.3189, Accuracy: 8758/10000 (88%) - lr=0.01, momentum=0.9 , epoch 1, batch_size = 128
# Test set: Average loss: 0.4987, Accuracy: 7927/10000 (79%) - lr=0.02, momentum=0.9 , epoch 1 , batch_size = 128
# Test set: Average loss: 1.8482, Accuracy: 3000/10000 (30%) - lr=0.03, momentum=0.9 , epoch 1 , batch_size = 128
# lr=0.01, momentum=0.9 , epoch 9
# accuracy at epoch 3
#Test set: Average loss: 0.0377, Accuracy: 9874/10000 (99%)

# accuracy starts decreasing for same parameters even after run time restart
#Test set: Average loss: 0.7211, Accuracy: 6943/10000 (69%) # lr=0.01, momentum=0.9 , epoch 4  , batch_size = 128

### changing batch size to 16

#Test set: Average loss: 1.4329, Accuracy: 4815/10000 (48%) # lr=0.01, momentum=0.9 , epoch 4  , batch_size = 16


epoch value is ,  1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-11-bc7f884a47af>:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return  F.log_softmax(output)   # return x for visualization
loss=0.4433377683162689 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.98it/s]



Test set: Average loss: 0.0609, Accuracy: 9794/10000 (98%)

epoch value is ,  2


loss=0.3998001515865326 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.71it/s]



Test set: Average loss: 0.0448, Accuracy: 9854/10000 (99%)

epoch value is ,  3


loss=0.35732924938201904 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.81it/s]



Test set: Average loss: 0.0426, Accuracy: 9840/10000 (98%)

epoch value is ,  4


loss=0.34913170337677 batch_id=227:  49%|████▊     | 228/469 [00:07<00:08, 28.65it/s]


KeyboardInterrupt: ignored